In [100]:
%%markdown

Getting Started


Getting Started


In [101]:
%%markdown

Import environment variables and session


Import environment variables and session


In [102]:
import os

from dotenv import load_dotenv
from snowflake.snowpark import Session
import snowflake.snowpark.functions as F 

In [103]:
load_dotenv()

account=os.environ.get("SNOWFLAKE_ACCOUNT")
user=os.environ.get("SNOWFLAKE_USER")
password=os.environ.get("SNOWFLAKE_PASSWORD")

connection_parameters = {
    "account": account,
    "user": user,
    "password": password,
    "role": "accountadmin",
    "database": "snowpark_demo_db",
    "schema": "snowpark_demo_schema"
}

session = Session.builder.configs(connection_parameters).create()


In [104]:
# Select CARTO data
# dataframe_carto_raw = session.table("analytics_toolbox.public.coord_sample")
# stored_proc_h3_boundary = builtin("H3_Boundary")

In [105]:
%%markdown

Snowflake dataframe


Snowflake dataframe


In [106]:
snow_df_spend = session.table("campaign_spend")
snow_df_revenue = session.table("monthly_revenue")

In [107]:
%%markdown

Total spend per month per channel


Total spend per month per channel


In [108]:
snow_df_spend_per_channel = snow_df_spend.group_by(F.year('DATE'), F.month('DATE'),'CHANNEL') \
    .agg(sum('TOTAL_COST').as_('TOTAL_COST')) \
    .with_column_renamed('"YEAR(DATE)"',"YEAR").with_column_renamed('"MONTH(DATE)"',"MONTH").sort('YEAR','MONTH')

In [109]:
print("Total spend per Year and Month For All Channels")
snow_df_spend_per_channel.show()

Total spend per Year and Month For All Channels
---------------------------------------------------
|"YEAR"  |"MONTH"  |"CHANNEL"      |"TOTAL_COST"  |
---------------------------------------------------
|2012    |5        |search_engine  |516431        |
|2012    |5        |video          |516729        |
|2012    |5        |email          |517208        |
|2012    |5        |social_media   |517618        |
|2012    |6        |video          |501098        |
|2012    |6        |search_engine  |506497        |
|2012    |6        |social_media   |504679        |
|2012    |6        |email          |501947        |
|2012    |7        |search_engine  |522780        |
|2012    |7        |email          |518405        |
---------------------------------------------------


In [110]:
%%markdown

# Total Spend Across All Channels

Let's further transform the campaign spend data so that each row represents total cost across all channels per year/month using the pivot() and sum() Snowpark DataFrame functions.
 
 This transformation lets us join with the revenue table so that our input features and target variable will be in a single table for model training.



# Total Spend Across All Channels

Let's further transform the campaign spend data so that each row represents total cost across all channels per year/month using the pivot() and sum() Snowpark DataFrame functions.
 
 This transformation lets us join with the revenue table so that our input features and target variable will be in a single table for model training.



In [111]:
snow_df_spend_per_month = snow_df_spend_per_channel \
    .pivot('CHANNEL', ['search_engine', 'social_media', 'video', 'email']) \
    .sum('TOTAL_COST').sort('YEAR', 'MONTH')

In [112]:
snow_df_spend_per_month = snow_df_spend_per_month.select(
    F.col("YEAR"),
    F.col("MONTH"),
    F.col("'search_engine'").as_("SEARCH_ENGINE"),
    F.col("'social_media'").as_("SOCIAL_MEDIA"),
    F.col("'video'").as_("VIDEO"),
    F.col("'email'").as_("EMAIL")
)

In [115]:
print("Total Spend Across All Channels")
snow_df_spend_per_month.show()


Total Spend Across All Channels
---------------------------------------------------------------------------
|"YEAR"  |"MONTH"  |"SEARCH_ENGINE"  |"SOCIAL_MEDIA"  |"VIDEO"  |"EMAIL"  |
---------------------------------------------------------------------------
|2012    |5        |516431           |517618          |516729   |517208   |
|2012    |6        |506497           |504679          |501098   |501947   |
|2012    |7        |522780           |521395          |522762   |518405   |
|2012    |8        |519959           |520537          |520685   |521584   |
|2012    |9        |507211           |507404          |511364   |507363   |
|2012    |10       |518942           |520863          |522768   |519950   |
|2012    |11       |505715           |505221          |505292   |503748   |
|2012    |12       |520148           |520711          |521427   |520724   |
|2013    |1        |522151           |518635          |520583   |521167   |
|2013    |2        |467736           |474679          |4

In [117]:
snow_df_revenue_per_month = snow_df_revenue.group_by('YEAR', 'MONTH').agg(sum('REVENUE')).sort('YEAR','MONTH').with_column_renamed('SUM(REVENUE)', 'REVENUE')

In [118]:
print("Total Revenue per Year and Month")
snow_df_revenue_per_month.show()

Total Revenue per Year and Month
---------------------------------
|"YEAR"  |"MONTH"  |"REVENUE"   |
---------------------------------
|2012    |5        |3264300.11  |
|2012    |6        |3208482.33  |
|2012    |7        |3311966.98  |
|2012    |8        |3311752.81  |
|2012    |9        |3208563.06  |
|2012    |10       |3334028.46  |
|2012    |11       |3185894.64  |
|2012    |12       |3334570.96  |
|2013    |1        |3316455.44  |
|2013    |2        |2995042.21  |
---------------------------------


In [119]:
%%markdown
 
 Join Total Spend and Total Revenue per Year and Month Across All Channels

Next let's join this revenue data with the transformed campaign spend data so that our input features (i.e. cost per channel) and target variable (i.e. revenue) can be loaded into a single table for model training.
    

 
 Join Total Spend and Total Revenue per Year and Month Across All Channels

Next let's join this revenue data with the transformed campaign spend data so that our input features (i.e. cost per channel) and target variable (i.e. revenue) can be loaded into a single table for model training.
    


In [120]:
snow_df_spend_and_revenue_per_month = snow_df_spend_per_month.join(snow_df_revenue_per_month, ["YEAR", "MONTH"])

In [121]:
session.close()